In [2]:
%pip install gurobipy

import gurobipy as gp
from gurobipy import GRB

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 60.3 MB/s eta 0:00:00


In [3]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import re

# --- 1. Leitura e Preparação dos Dados ---

def ler_dados_dgdp(nome_arquivo):
    with open(nome_arquivo, 'r') as f:
        content = f.read()

    # Expressão regular para extrair a matriz de distâncias (d_inf e d_sup)
    def extract_matrix(text, param_name):
        # Localiza o bloco de dados da matriz
        pattern = rf'param {param_name}:\s*1\s*2\s*3\s*4\s*5\s*:=\s*(.*?);'
        match = re.search(pattern, text, re.DOTALL)
        if not match:
            raise ValueError(f"Bloco 'param {param_name}:' não encontrado.")

        data_block = match.group(1).strip()

        # Processa as linhas da matriz
        matrix_data = {}
        lines = data_block.split('\n')
        for line in lines:
            parts = line.strip().split()
            if len(parts) == 6: # 1 (índice) + 5 (valores)
                i = int(parts[0])
                # Mapeia para float
                matrix_data[i] = {j: float(parts[j]) for j in range(1, 6)}

        return pd.DataFrame.from_dict(matrix_data, orient='index')

    D_inf = extract_matrix(content, 'di')
    D_sup = extract_matrix(content, 'ds')

    # Extrair n (já sabemos que n=5)
    n = 5

    return n, D_inf, D_sup

# Carregar os dados
try:
    N_dgdp, D_inf, D_sup = ler_dados_dgdp('tc2exemplon5t.dat')
    I_dgdp = range(1, N_dgdp + 1)
    J_dgdp = range(1, N_dgdp + 1)

    # Definir as constantes (Arbitrárias, pois não foram fornecidas no prompt)
    C_capacidade = 2   # Capacidade de cada local
    F_custo_fixo = 10 # Custo fixo de abertura de cada local (Um valor maior para dar peso na FO)

except Exception as e:
    print(f"Erro ao ler os dados do DGDP: {e}")
    # Criar dados fictícios para continuar com o modelamento se a leitura falhar
    N_dgdp = 5
    I_dgdp = range(1, N_dgdp + 1)
    J_dgdp = range(1, N_dgdp + 1)
    C_capacidade = 2
    F_custo_fixo = 10
    # Matrizes D_inf e D_sup (usando valores do snippet)
    data_inf = [[0.000000, 1.000000, 1.682942, 2.580498, 3.004056],
                [1.000000, 0.000000, 1.000000, 1.682942, 2.061064],
                [1.682942, 1.000000, 0.000000, 1.000000, 1.682942],
                [2.580498, 1.682942, 1.000000, 0.000000, 1.000000],
                [3.004056, 2.061064, 1.682942, 1.000000, 0.000000]]
    D_inf = pd.DataFrame(data_inf, index=I_dgdp, columns=J_dgdp)
    data_sup = [[0.000000, 1.150000, 1.832942, 2.730498, 3.154056],
                [1.150000, 0.000000, 1.150000, 1.832942, 2.211064],
                [1.832942, 1.150000, 0.000000, 1.150000, 1.832942],
                [2.730498, 1.832942, 1.150000, 0.000000, 1.150000],
                [3.154056, 2.211064, 1.832942, 1.150000, 0.000000]]
    D_sup = pd.DataFrame(data_sup, index=I_dgdp, columns=J_dgdp)

# --- 2. Modelagem do Problema no Gurobi ---

try:
    model_dgdp = gp.Model("DGDP_Cetica")

    # --- Variáveis de Decisão ---
    # x_ij: Alocação do cliente i ao local j (Binária)
    x_dgdp = model_dgdp.addVars(I_dgdp, J_dgdp, vtype=GRB.BINARY, name="x_dgdp")

    # y_j: Local j é aberto (Binária)
    y_dgdp = model_dgdp.addVars(J_dgdp, vtype=GRB.BINARY, name="y_dgdp")

    # theta: Custo de alocação máximo (Contínua)
    theta = model_dgdp.addVar(vtype=GRB.CONTINUOUS, name="theta")

    # --- Função Objetivo ---
    # min Z = sum(F_j * y_j) + theta
    model_dgdp.setObjective(F_custo_fixo * gp.quicksum(y_dgdp[j] for j in J_dgdp) + theta, GRB.MINIMIZE)

    # --- Restrições ---

    # 1. Alocação Única por Cliente
    model_dgdp.addConstrs((gp.quicksum(x_dgdp[i, j] for j in J_dgdp) == 1 for i in I_dgdp), name="AlocacaoUnica_dgdp")

    # 2. Capacidade do Local (Alocação só em locais abertos e capacidade respeitada)
    model_dgdp.addConstrs((gp.quicksum(x_dgdp[i, j] for i in I_dgdp) <= C_capacidade * y_dgdp[j] for j in J_dgdp), name="Capacidade_dgdp")

    # 3. Restrição Max-Min (Theta maior ou igual ao custo de alocação na pior condição para cada cliente)
    model_dgdp.addConstrs((theta >= gp.quicksum(D_sup.loc[i, j] * x_dgdp[i, j] for j in J_dgdp) for i in I_dgdp), name="MaxMin_dgdp")

    # --- Otimização ---
    model_dgdp.optimize()

    # --- 3. Apresentação dos Resultados ---

    if model_dgdp.status == GRB.OPTIMAL:
        custo_fixo_total = sum(F_custo_fixo * y_dgdp[j].X for j in J_dgdp)
        custo_alocacao_max = theta.X
        custo_total = model_dgdp.ObjVal

        locais_abertos = [j for j in J_dgdp if y_dgdp[j].X > 0.5]
        alocacoes = [(i, j, x_dgdp[i, j].X) for i in I_dgdp for j in J_dgdp if x_dgdp[i, j].X > 0.5]

        print("\n--- Resultados (Problema 2 - DGDP Cético) ---")
        print(f"Status da Otimização: Ótimo")
        print(f"Custo Total Mínimo (Pior Condição): {custo_total:.4f}")
        print(f"Custo Fixo Total (sum(F_j * y_j)): {custo_fixo_total:.4f}")
        print(f"Custo Máximo de Alocação (theta): {custo_alocacao_max:.4f}")
        print(f"Número de Locais Abertos: {len(locais_abertos)}")
        print(f"Locais Abertos: {locais_abertos}")

        print("\nAlocações (Cliente, Local):")
        for i, j, val in alocacoes:
            print(f"Cliente {i} alocado ao Local {j} (Distância: [{D_inf.loc[i, j]:.4f}, {D_sup.loc[i, j]:.4f}])")

    else:
        print(f"\nOtimização não Convergiu para um Resultado Ótimo. Status: {model_dgdp.status}")

except gp.GurobiError as e:
    print(f"Erro do Gurobi: {e.message}")
except Exception as e:
    print(f"Ocorreu um erro: {e}")

Erro ao ler os dados do DGDP: [Errno 2] No such file or directory: 'tc2exemplon5t.dat'
Restricted license - for non-production use only - expires 2026-11-23
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 15 rows, 31 columns and 80 nonzeros
Model fingerprint: 0x01ec4f50
Variable types: 1 continuous, 30 integer (30 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+00, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 42.2110640
Presolve time: 0.00s
Presolved: 15 rows, 31 columns, 80 nonzeros
Variable types: 1 continuous, 30 integer (30 binary)

Root relaxation: objective 2.500000e+01, 14 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bou

In [11]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import numpy as np
import re
from google.colab import files
from google.colab import drive

# =================================================================
# BLOCO DE CONFIGURAÇÃO E TRATAMENTO DE ARQUIVOS
# =================================================================

# 1. Monta o Drive
print("Montando o Google Drive (necessário para persistência de arquivos)...")
drive.mount('/content/drive', force_remount=True)

# 2. Caminho de busca (MANTENHA APENAS UM CAMINHO ATIVO)
CAMINHO_ARQUIVO = 'tc2exemplon5t.dat'
# Se o arquivo estiver em uma pasta específica do Drive (exemplo: PastaDoCurso):
# CAMINHO_ARQUIVO = '/content/drive/MyDrive/PastaDoCurso/tc2exemplon5t.dat'

# 3. Bloco de tratamento de erro para garantir que o arquivo seja acessível
arquivo_acessivel = False
try:
    with open(CAMINHO_ARQUIVO, 'r'):
        arquivo_acessivel = True
        print(f"Arquivo '{CAMINHO_ARQUIVO}' encontrado e acessível.")
except FileNotFoundError:
    print(f"\nAVISO: Arquivo '{CAMINHO_ARQUIVO}' não encontrado no caminho fornecido.")

if not arquivo_acessivel:
    print("\n--- INSTRUÇÃO DE UPLOAD ---")
    print("Por favor, selecione o arquivo 'tc2exemplon5t.dat' para fazer o upload na sessão atual do Colab:")
    # O upload salva o arquivo no diretório de trabalho padrão (que é onde o código procurará por 'tc2exemplon5t.dat')
    uploaded = files.upload()
    if 'tc2exemplon5t.dat' in uploaded:
        CAMINHO_ARQUIVO = 'tc2exemplon5t.dat'
        print(f"Upload de '{CAMINHO_ARQUIVO}' realizado com sucesso.")
    else:
        raise FileNotFoundError("O arquivo necessário não foi carregado.")

# =================================================================
# FUNÇÕES DE LEITURA
# =================================================================

def ler_dados_dgdp(nome_arquivo):
    with open(nome_arquivo, 'r') as f:
        content = f.read()

    N_dgdp_static = 5

    def extract_matrix(text, param_name):
        pattern = rf'param {param_name}:\s*1\s*2\s*3\s*4\s*5\s*:=\s*(.*?);'
        match = re.search(pattern, text, re.DOTALL)
        if not match:
             match = re.search(rf'param {param_name}:\s*.*?:=\s*(.*?);', text, re.DOTALL)
             if not match:
                 raise ValueError(f"Bloco 'param {param_name}:' não encontrado.")

        data_block = match.group(1).strip()

        matrix_data = {}
        lines = data_block.split('\n')
        for line in lines:
            parts = line.strip().split()
            if len(parts) == 6:
                i = int(parts[0])
                matrix_data[i] = {j: float(parts[j]) for j in range(1, N_dgdp_static + 1)}

        return pd.DataFrame.from_dict(matrix_data, orient='index', columns=range(1, N_dgdp_static + 1))

    D_inf = extract_matrix(content, 'di')
    D_sup = extract_matrix(content, 'ds')

    n = N_dgdp_static

    return n, D_inf, D_sup

# =================================================================
# EXECUÇÃO E MODELAGEM (PROBLEMA 2)
# =================================================================

# Carregar os dados (Problema 2)
N_dgdp, D_inf, D_sup = ler_dados_dgdp(CAMINHO_ARQUIVO)
I_dgdp = range(1, N_dgdp + 1)
J_dgdp = range(1, N_dgdp + 1)

# Definir as constantes (Valores assumidos)
C_capacidade = 2
F_custo_fixo = 10.0


print("\nIniciando otimização para o Problema 2 (DGDP Cético)...")

try:
    model_dgdp = gp.Model("DGDP_Cetica")

    # --- Variáveis de Decisão ---
    x_dgdp = model_dgdp.addVars(I_dgdp, J_dgdp, vtype=GRB.BINARY, name="x_dgdp")
    y_dgdp = model_dgdp.addVars(J_dgdp, vtype=GRB.BINARY, name="y_dgdp")
    theta = model_dgdp.addVar(vtype=GRB.CONTINUOUS, name="theta")

    # --- Função Objetivo ---
    model_dgdp.setObjective(F_custo_fixo * gp.quicksum(y_dgdp[j] for j in J_dgdp) + theta, GRB.MINIMIZE)

    # --- Restrições ---
    model_dgdp.addConstrs((gp.quicksum(x_dgdp[i, j] for j in J_dgdp) == 1 for i in I_dgdp), name="AlocacaoUnica_dgdp")
    model_dgdp.addConstrs((gp.quicksum(x_dgdp[i, j] for i in I_dgdp) <= C_capacidade * y_dgdp[j] for j in J_dgdp), name="Capacidade_dgdp")
    model_dgdp.addConstrs((theta >= gp.quicksum(D_sup.loc[i, j] * x_dgdp[i, j] for j in J_dgdp) for i in I_dgdp), name="MaxMin_dgdp")

    # --- Otimização ---
    model_dgdp.optimize()

    # --- Apresentação dos Resultados ---

    print("\n" + "="*50)
    print("Resultados da Otimização - Problema 2 (DGDP Cético)")
    print("="*50)

    if model_dgdp.status == GRB.OPTIMAL:
        custo_fixo_total = sum(F_custo_fixo * y_dgdp[j].X for j in J_dgdp)
        custo_alocacao_max = theta.X
        custo_total = model_dgdp.ObjVal

        locais_abertos = [j for j in J_dgdp if y_dgdp[j].X > 0.5]
        alocacoes = [(i, j, x_dgdp[i, j].X) for i in I_dgdp for j in J_dgdp if x_dgdp[i, j].X > 0.5]

        print(f"Status da Otimização: Ótimo")
        print(f"Custo Total Mínimo (Pior Condição): {custo_total:.4f}")
        print(f"Custo Fixo Total (sum(F_j * y_j)): {custo_fixo_total:.4f}")
        print(f"Custo Máximo de Alocação (theta): {custo_alocacao_max:.4f}")
        print(f"Número de Locais Abertos: {len(locais_abertos)}")
        print(f"Locais Abertos: {locais_abertos}")

        print("\nAlocações (Cliente, Local):")
        for i, j, val in alocacoes:
            print(f"Cliente {i} alocado ao Local {j} (Distância: [{D_inf.loc[i, j]:.4f}, {D_sup.loc[i, j]:.4f}])")

    else:
        print(f"\nOtimização não Convergiu para um Resultado Ótimo. Status: {model_dgdp.status}")

except gp.GurobiError as e:
    print(f"Erro do Gurobi: {e.message}")
except Exception as e:
    print(f"Ocorreu um erro: {e}")

Montando o Google Drive (necessário para persistência de arquivos)...
Mounted at /content/drive

AVISO: Arquivo 'tc2exemplon5t.dat' não encontrado no caminho fornecido.

--- INSTRUÇÃO DE UPLOAD ---
Por favor, selecione o arquivo 'tc2exemplon5t.dat' para fazer o upload na sessão atual do Colab:


Saving tc2exemplon5t.dat to tc2exemplon5t.dat
Upload de 'tc2exemplon5t.dat' realizado com sucesso.

Iniciando otimização para o Problema 2 (DGDP Cético)...
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 15 rows, 31 columns and 80 nonzeros
Model fingerprint: 0x01ec4f50
Variable types: 1 continuous, 30 integer (30 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+00, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 42.2110640
Presolve time: 0.00s
Presolved: 15 rows, 31 columns, 80 nonzeros
Variable types: 1 continuous, 30 integer (30 binary)

Root relaxation: objective 2.500000e+01, 14 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Boun